In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
%matplotlib inline

In [2]:
train = pd.read_csv("C:\\Users\\user\\mercedes-benz-greener-manufacturing\\train.csv")
test =  pd.read_csv("C:\\Users\\user\\mercedes-benz-greener-manufacturing\\test.csv")

In [3]:
# store it as Y
Y_train = train["y"]
train.drop(["y"], axis=1, inplace=True)
id_test = test["ID"]
train.describe()

,ID,X10,X11,X12,X13,X14,X15,X16,X17,X18,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
count,4209.000000,4209.000000,4209.0,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,...,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000
mean,4205.960798,0.013305,0.0,0.075077,0.057971,0.428130,0.000475,0.002613,0.007603,0.007840,...,0.318841,0.057258,0.314802,0.020670,0.009503,0.008078,0.007603,0.001663,0.000475,0.001426
std,2437.608688,0.114590,0.0,0.263547,0.233716,0.494867,0.021796,0.051061,0.086872,0.088208,...,0.466082,0.232363,0.464492,0.142294,0.097033,0.089524,0.086872,0.040752,0.021796,0.037734
min,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2095.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4220.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,6314.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,8417.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
train.describe(include=["O"])

,X0,X1,X2,X3,X4,X5,X6,X8
count,4209,4209,4209,4209,4209,4209,4209,4209
unique,47,27,44,7,4,29,12,25
top,z,aa,as,c,d,w,g,j
freq,360,833,1659,1942,4205,231,1042,277


In [5]:
num_train = len(train)
all_data = pd.concat([train, test])

In [6]:
all_data.drop(["ID"], axis=1, inplace=True)
all_data = pd.get_dummies(all_data)

In [7]:
X_train = all_data[:num_train]
X_test = all_data[num_train:]

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split( X_train, Y_train, test_size = 0.3, random_state = 100)

In [9]:
clf_tuned = DecisionTreeRegressor(max_depth=6)

In [10]:
clf_tuned.fit(X_train, y_train)
y_pred = clf_tuned.predict(X_cv)

In [11]:
from sklearn.metrics import r2_score
print(r2_score(y_pred, y_cv))

0.0707479864028


In [12]:
d = []
# Decision tree tuning
for crtr in ['mse']:
    for md in [7,8,9,10,11,12,13]:
        for spltr in ['best','random']:
            for mss in [5,10,15,25,40,65,105]:
                for msl in [5,10,15,25,40,65,105]:
                    dts = DecisionTreeRegressor(criterion=crtr, max_depth=md,
                                max_features=None, max_leaf_nodes=None, min_samples_leaf=msl,
                                min_samples_split=mss, min_weight_fraction_leaf=0.0,
                                presort=False, random_state=100, splitter=spltr)
                    dts.fit(X_train, y_train)
                    y_pred = dts.predict(X_cv)
                    sip=r2_score(y_pred,y_cv)
                    #print("score for {} criterion, {} max_depth, {} splitter, {} min_samples_split, {} min_samples_leaf is {}".format(crtr,md,spltr,mss,msl,sip))
                    d.append({'criterion': crtr, 'max_depth': md, 'splitter':spltr , 'min_samples_split' :mss, 'min_samples_leaf' : msl, 'score' : sip})

In [13]:
d=pd.DataFrame(d)

In [14]:
d.to_csv('bestmodel2.csv', index=False) # LB : 0.74163

In [21]:
dtr_tuned = DecisionTreeRegressor(criterion='mse', max_depth=9,
                                max_features=None, max_leaf_nodes=None, min_samples_leaf=25,
                                min_samples_split=105, min_weight_fraction_leaf=0.0,
                                presort=False, random_state=100, splitter='random')

In [22]:
dtr_tuned.fit(X_train, y_train)
y_test = dtr_tuned.predict(X_test)
submission = pd.DataFrame({
        "ID": id_test,
        "y": y_test
    })
submission.to_csv('tunedtree.csv', index=False)

In [23]:
# This statement builds a dot file.
from sklearn import tree
from sklearn.tree import export_graphviz
cols = list(X_train.columns.values)
tree.export_graphviz(dtr_tuned, out_file='tunedtree.dot',feature_names  = cols) 